In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 8 --executor-memory 5g --executor-cores 4 --driver-memory 8g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
from pyspark.sql.functions import split, col, explode, regexp_replace, round,when, collect_list, sum, count, array, row_number, avg
from pyspark.ml.linalg import Vectors
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [3]:
all = spark.read.parquet("all")

In [38]:
all_feature.select("user_id", "item_id").distinct().count()

5032624

In [39]:
df.count()

5032624

## Загружаем train и test наборы данных

In [4]:

schema1 = StructType() \
      .add("user_id",IntegerType(),True) \
      .add("item_id",IntegerType(),True) \
      .add("purchase",IntegerType(),True) 
      



In [5]:

schema_test = StructType() \
      .add("user_id",IntegerType(),True) \
      .add("item_id",IntegerType(),True) 
      
      

In [6]:
 
df = spark.read.options(delimiter=',').options(header = True).schema(schema1).csv("/labs/slaba03/laba03_train.csv")


In [7]:

df_test = spark.read.options(delimiter=',').options(header = True).schema(schema_test).csv("/labs/slaba03/laba03_test.csv")
      
      

## Обрабатываем таблицу с жанрами (делаем explode)

### Исключаем наиболее популярный жанр - "Зарубежные" т.к это улучшает метрику модели.
### Первоначально была попытка склеить жанры, но она не улучшила метрику

In [9]:
df2 = spark.read.options(delimiter='\t').options(header = True).csv("/labs/slaba03/laba03_items.csv")

#df2_ = df2.withColumn("genreArray",explode(split(regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(col("genres"),"Фантастичские","Фантастика"),"Триллеры","Триллер"),"Спортивные","Спорт"),"Семейные","Семейный"),"Приключения","Приключение"),"Мультфильмы","Мультфильм"),"Музыкальные","Музыкальный"),"Мелодрамы","Мелодрама"),"Короткометражки","Короткометражные"),"Комедии","Комедия"),"Исторические","Исторический"),"Драмы","Драма"),"Документальные","Документальный"),"Военные","Военный")," сказка","Сказки"),"Арт-хаус","Артхаус"),"Боевики","Боевик"),","))).select("item_id","genreArray")
# Первоначально была попытка склеить жанры, но она не улучшила метрику


df2_ = df2.withColumn("genreArray",explode(split("genres",","))) \
          .filter(col("content_type") == 1) \
          .filter((col("genreArray") != 'Зарубежные')) \
          .select("item_id","genreArray") \
          .fillna("жанр не указан")


## Соединяем train и test массивы с полученной таблицей с жанрами

In [10]:


df3 = df.join(df2_,df2_.item_id == df.item_id,"left") \
        .select("user_id",df.item_id,"genreArray","purchase") \
        .fillna(0)

In [ ]:
df3 = df.join(df2_,df2_.item_id == df.item_id,"left") \
        .select("user_id",df.item_id,"genreArray","purchase","decade") \
        .fillna(0)

In [11]:
df3_test = df_test.join(df2_,df2_.item_id == df_test.item_id,"left") \
                  .select("user_id",df_test.item_id,"genreArray") \
                  .fillna(0)

## По каждому жанру считаем кол-во покупок и не покупок

In [12]:
df7 = df3.groupBy("genreArray") \
         .pivot("purchase") \
         .count() \
         .fillna(0) \
         .select("genreArray",col("1").alias("s7"), col("0").alias("c7"))

## Строим агрегат по клиентам и жанрам (кол-во покупок и общее кол-во) и джойним с общим кол-вом покупок и не покупок на жанр

In [13]:

df3_purchase = df3.groupBy("user_id","genreArray") \
                  .agg(sum("purchase").alias("s1"), count("purchase").alias("c1")) \
                  .join(df7,["genreArray"],"inner")

## Для train и test cтроим первоначальную таблицу с фичами (для каждой пары user_id, item_id) считаем сколько фильмов аналогичных жанров купил и не купил клиент, а также сколько фильмов аналогичных жанров было куплено и не куплено в целом)

In [14]:
df3_feature = df3 \
          .join(df3_purchase,(df3_purchase.genreArray == df3.genreArray) & (df3.user_id == df3_purchase.user_id) ,"left") \
          .select(df3.user_id,df3.genreArray,"item_id","purchase",col("s1"),(col("c1") - col("s1")).alias("c"),"s7","c7") \
          .fillna(0) \
          .groupBy("user_id","item_id","purchase") \
          .agg( \
               sum("s1").alias("s") \
              ,sum("c").alias("c") \
              ,sum("s7").alias("s7") \
              ,sum("c7").alias("c7")) 
          

In [15]:
df3_feature_test = df3_test \
    .join(df3_purchase,(df3_purchase.genreArray == df3_test.genreArray) & (df3_test.user_id == df3_purchase.user_id) ,"left") \
    .select(df3_test.user_id,df3_test.genreArray,"item_id",col("s1"),(col("c1") - col("s1")).alias("c"),"s7","c7") \
    .fillna(0) \
    .groupBy("user_id","item_id") \
    .agg( \
        sum("s1") .alias("s") \
       ,sum("c").alias("c") \
       ,sum("s7").alias("s7") \
       ,sum("c7").alias("c7")) 

## Год выпуска контента делим на категории (экспертно) < 1930, < 1950, < 1970, < 1980, < 1990, < 2000, < 2010, < 2015, < 2020, > 2020

In [16]:
df2_cat = df2.withColumn("decade",when(df2.year < 1930, 1) \
                                 .when(df2.year < 1950, 2) \
                                 .when(df2.year < 1970, 3) \
                                 .when(df2.year < 1980, 4) \
                                 .when(df2.year < 1990, 5) \
                                 .when(df2.year < 2000, 6) \
                                 .when(df2.year < 2010, 7) \
                                 .when(df2.year < 2015, 8) \
                                 .when(df2.year < 2020, 9) \
                                 .when(df2.year >= 2020, 10) \
                                 .otherwise(-10)) \
             .select("item_id","decade","year","genres") \
             .fillna(0)

## Train и test соединяем с таблицей категорий по году выпуска контента

In [17]:
df3_cat = df.join(df2_cat,df2_cat.item_id == df.item_id,"inner") \
            .select("user_id",df.item_id,"decade","purchase","year")

In [18]:
df3_cat_test = df_test.join(df2_cat,df2_cat.item_id == df_test.item_id,"inner") \
                      .select("user_id",df_test.item_id,"decade","year")

## Для каждого клиента и категории считаем кол-во покупок и общее кол-во

In [19]:
df3_cat_purchase = df3_cat.groupBy("user_id","decade") \
                          .agg(sum("purchase").alias("s2"), count("purchase").alias("c2"))

In [46]:
df3_item_cat_purchase = df3_cat.groupBy("item_id","decade") \
                          .agg(sum("purchase").alias("s2i"), count("purchase").alias("c2i"))

## Для train и test cтроим таблицу с фичами по категории по году выпуска контента (для каждой пары user_id, item_id) считаем сколько фильмов аналогичных категорий купил и не купил клиент)

In [20]:
df3_cat_feature = df3_cat \
   .join(df3_cat_purchase,(df3_cat_purchase.decade == df3_cat.decade)  \
               & (df3_cat.user_id == df3_cat_purchase.user_id) ,"left") \
   .select(df3_cat.user_id,df3_cat.decade,"item_id","purchase",col("s2"),(col("c2") - col("s2")).alias("c2")) \
   .groupBy("user_id","item_id","purchase") \
   .agg( \
        sum("s2").alias("s2") \
       ,sum("c2").alias("c2")) \
   .fillna(0)

In [21]:
df3_cat_feature_test = df3_cat_test \
 .join(df3_cat_purchase,(df3_cat_purchase.decade == df3_cat_test.decade) \
                  & (df3_cat_test.user_id == df3_cat_purchase.user_id) ,"left") \
 .select(df3_cat_test.user_id,df3_cat_test.decade,"item_id",col("s2"),(col("c2") - col("s2")).alias("c2")) \
 .fillna(0) \
 .groupBy("user_id","item_id") \
 .agg( \
    sum("s2").alias("s2") \
   ,sum("c2").alias("c2")) 

## Соединяем фичи по жанрам и фичи по году выпуска контента

In [22]:
df3_feature2 = df3_feature \
  .join(df3_cat_feature,(df3_feature.user_id == df3_cat_feature.user_id) \
                    & (df3_feature.item_id == df3_cat_feature.item_id) ,"left") \
  .select(df3_feature.user_id,df3_feature.purchase,df3_feature.item_id,"s","c",(col("s")/(col("c") + col("s"))).alias("z") \
        ,"s2","c2", (col("s2")/(col("c2") + col("s2"))).alias("z2") \
        ,"s7","c7", (col("s7")/(col("c7") + col("s7"))).alias("z7")) \
  .fillna(0)


In [23]:
df3_feature2_test = df3_feature_test \
  .join(df3_cat_feature_test,(df3_feature_test.user_id == df3_cat_feature_test.user_id) \
     & (df3_feature_test.item_id == df3_cat_feature_test.item_id) ,"left") \
  .select(df3_feature_test.user_id,df3_feature_test.item_id,"s","c",(col("s")/(col("c") + col("s"))).alias("z") \
          ,"s2","c2", (col("s2")/(col("c2") + col("s2"))).alias("z2") \
          ,"s7","c7", (col("s7")/(col("c7") + col("s7"))).alias("z7")) \
 .fillna(0)

## Для каждого клиента строим абсолютное кол-во покупок и не покупок и относительное кол-во покупок и добавляем их к фичам

In [24]:
df_f = df.groupBy("user_id") \
         .pivot("purchase") \
         .count() \
         .fillna(0) \
         .select("user_id",col("1").alias("su"), col("0").alias("cu"), (col("1")/(col("0") + col("1"))).alias("zu"))

In [25]:
df3_feature3 = df3_feature2.join(df_f,["user_id"],"left").fillna(0)

In [26]:
df3_feature3_test = df3_feature2_test.join(df_f,["user_id"],"left").fillna(0) 

## Для каждого фильма строим абсолютное кол-во покупок и не покупок и относительное кол-во покупок и добавляем их к фичам

In [27]:
df_i = df.groupBy("item_id") \
         .pivot("purchase") \
         .count() \
         .fillna(0) \
         .select("item_id",col("1").alias("si"), col("0").alias("ci"),  (col("1")/(col("0") + col("1"))).alias("zi") ) \

In [28]:
df3_feature4 = df3_feature3.join(df_i,["item_id"],"left").fillna(0)

In [29]:
df3_feature4_test = df3_feature3_test.join(df_i,["item_id"],"left").fillna(0)

## Загружаем датасет с инофрмацией о просмотрах

In [30]:
df4 = spark.read.options(delimiter=',').options(header = True).csv("/labs/slaba03/laba03_views_programmes.csv")

## Для каждого клиента строим фичи по среднему и абсолютному времени просмотра и кол-ву просмотров и добавляем их к общей таблице с фичами

In [31]:
df5 = df4.select("user_id", (df4.ts_end - df4.ts_start).alias("t")) \
         .groupBy("user_id") \
         .agg(avg("t").alias("al5"), count("t").alias("cl5"), sum("t").alias("sl5"))  

In [32]:
df3_feature5 = df3_feature4.join(df5,["user_id"],"left").fillna(0)

In [33]:
df3_feature5_test = df3_feature4_test.join(df5,["user_id"],"left").fillna(0)

## Получаем общую таблицу с фичами (строим 2 производные фичи - оносительное кол-во покупок по жанрам и году выпуска)

In [34]:
all_feature = df3_feature5.withColumn("sf",col("s")/(col("su")+ col("cu"))) \
                          .withColumn("s2f",col("s2")/(col("su")+ col("cu"))) \
                          .fillna(0) \
                          .cache()  


In [35]:
all_feature_test = df3_feature5_test.withColumn("sf",col("s")/(col("su")+ col("cu"))) \
                                    .withColumn("s2f",col("s2")/(col("su")+ col("cu"))) \
                                    .fillna(0) \
                                    .cache() 


## Собираем фичи в вектор (берем только улучшающие метрику модели)

In [279]:
from pyspark.ml.feature import VectorAssembler



'''assembler = VectorAssembler(
    inputCols=["s", "c","z", "s2", "c2", "z2","su","cu","zu","si","ci","zi","al5","cl5","sl5", "c7","sf","s2f"],
    outputCol="features") '''


assembler = VectorAssembler(
    inputCols=["s", "c", "z","s2","su","zu","si","ci","zi","cl5","sl5","c7","sf","s2f","sd","zd","sdf"],
    outputCol="features")

assembler = VectorAssembler(
    inputCols=["s", "c", "z","s2","su","zu","si","ci","zi","cl5","sl5","c7","sf","s2f"],
    outputCol="features")



## Логистическая регрессия 

In [280]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol= "purchase", maxIter=6, regParam=0.001)

   

In [281]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[assembler,lr])

In [282]:
model = pipeline.fit(all_feature)

## Обработка результата модели и выгрузка в файл

In [283]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

firstelement=udf(lambda v:float(v[1]),FloatType())

In [284]:
prediction = model.transform(all_feature_test).cache()



In [285]:
from pyspark.sql import Window
w = Window.orderBy("user_id","item_id")
final = prediction.select((row_number().over(w)).alias("r"),"user_id","item_id",firstelement("probability").alias("purchase")).coalesce(1)


In [286]:
!hdfs dfs -rm -R lab3.csv

22/10/29 22:47:15 INFO fs.TrashPolicyDefault: Moved: 'hdfs://spark-master-1.newprolab.com:8020/user/yuriy.severyukhin/lab3.csv' to trash at: hdfs://spark-master-1.newprolab.com:8020/user/yuriy.severyukhin/.Trash/Current/user/yuriy.severyukhin/lab3.csv1667072835434


In [287]:
final.write.options(header='True', delimiter=',',mode = 'overwrite').csv("lab3.csv")

In [288]:
 !hdfs dfs -get lab3.csv/*

get: `_SUCCESS': File exists


In [289]:
!rm lab03.csv

In [290]:
!mv part*.csv lab03.csv

In [40]:
spark.stop()